# Clasificadores de sesgo

## Imports

In [ ]:
# ============================
# Configuración de entorno
# ============================
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"  # Evita warnings de paralelismo en tokenizers de HuggingFace

# ============================
# Librerías Numéricas y DataFrames
# ============================
import numpy as np  # Computación numérica
import pandas as pd  # Manipulación de datos tabulares

# ============================
# Deep Learning y manejo de modelos
# ============================
import torch  # PyTorch: Deep learning y tensor computations
import torch.nn as nn  # Módulos de redes neuronales en PyTorch
from torch.utils.data import Dataset, DataLoader  # Utilidades para datasets y dataloaders en PyTorch
from torch.optim import AdamW  # Optimizador AdamW de PyTorch
from torch.amp import autocast, GradScaler  # Mixed precision training

# ============================
# Modelos preentrenados y NLP
# ============================
from transformers import AutoTokenizer, AutoModel  # Tokenización y modelos preentrenados de transformers (HuggingFace)

# ============================
# Ciencia de Datos y Machine Learning
# ============================
from sklearn.preprocessing import StandardScaler  # Normalización de datos
from sklearn.model_selection import (
    StratifiedKFold,  # K-Fold estratificado
    KFold,            # K-Fold simple
    ParameterGrid     # Grid de parámetros para búsqueda de hiperparámetros
)
from sklearn.metrics import accuracy_score, f1_score  # Métricas de evaluación
from sklearn.utils.class_weight import compute_class_weight  # Cálculo de pesos de clase

# ============================
# Muestreo y balanceo de clases
# ============================
from imblearn.combine import SMOTETomek, SMOTEENN  # Técnicas combinadas de sobremuestreo y submuestreo
from imblearn.over_sampling import RandomOverSampler  # Sobremuestreo aleatorio
from imblearn.under_sampling import RandomUnderSampler  # Submuestreo aleatorio
from imblearn.pipeline import Pipeline  # Pipelines de balanceo

# ============================
# Utilidades varias
# ============================
import json  # Manejo de archivos y datos JSON
from tqdm import tqdm  # Barra de progreso para loops
from collections import defaultdict  # Diccionario con valores por defecto
import os.path  # Operaciones de sistema de archivos
import warnings  # Manejo de warnings

# ============================
# (Opcional) Otros modelos ML
# ============================
import lightgbm as lgb  # Gradient boosting, solo si se usa LightGBM

## Funciones generales y definiciones

### Varables globales


In [ ]:
# =================== CONFIGURACIÓN ===================

# This variables need to be adjusted depending on the model and task

MODEL_NAME = "PlanTL-GOB-ES/roberta-base-bne"
SEED = 42
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_CLASSES = 2 
MAX_LEN = 128
BATCH_SIZE = 256
GRADIENT_ACCUMULATION_STEPS = 2
USE_FP16 = True

torch.manual_seed(SEED)
np.random.seed(SEED)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

### Clase y funciones de entrenamiento


In [ ]:
# Training model

class TextTabularDataset(Dataset):
    """
    Dataset para tareas que combinan texto (tokenizado) y variables tabulares.
    """

    def __init__(self, input_ids, attention_mask, tabular_data, labels=None):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.tabular_data = tabular_data
        self.labels = labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        inputs = {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'tabular': torch.tensor(self.tabular_data[idx], dtype=torch.float32)
        }
        if self.labels is not None:
            return inputs, torch.tensor(self.labels[idx], dtype=torch.long)
        return inputs


class RobertaWithTabularDeepHead(nn.Module):
    """
    Modelo que combina las salidas de un modelo Roberta/BERT y variables tabulares adicionales, 
    seguido por una cabeza densa profunda.
    """

    def __init__(self, roberta_model, tabular_dim=1, num_classes=3, dropout=0.2):
        super().__init__()
        self.roberta = AutoModel.from_pretrained(roberta_model)
        concat_dim = self.roberta.config.hidden_size + tabular_dim
        self.head = nn.Sequential(
            nn.Linear(concat_dim, 128),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(128, 32),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(32, num_classes)
        )

    def forward(self, input_ids, attention_mask, tabular):
        outputs = self.roberta(input_ids=input_ids,
                               attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0]  # [CLS] token
        concat = torch.cat([pooled_output, tabular], dim=1)
        return self.head(concat)

# Training function

def train_and_validate(
    model, train_loader, val_loader, class_weights,
    epochs=5, lr=2e-5, patience=2
):
    model.to(DEVICE)
    optimizer = AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    if USE_FP16:
        try:
            scaler = GradScaler(device_type="cuda")
        except TypeError:
            scaler = GradScaler()
    best_val_f1_macro = -1
    best_val_acc = 0
    best_val_f1_per_class = None
    best_state_dict = None
    patience_counter = 0

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        for step, batch in enumerate(train_loader):
            inputs, labels = batch
            labels = labels.to(DEVICE)
            with autocast(device_type="cuda", enabled=USE_FP16):
                outputs = model(
                    input_ids=inputs['input_ids'].to(DEVICE),
                    attention_mask=inputs['attention_mask'].to(DEVICE),
                    tabular=inputs['tabular'].to(DEVICE)
                )
                loss = criterion(outputs, labels) / GRADIENT_ACCUMULATION_STEPS
            if USE_FP16:
                scaler.scale(loss).backward()
            else:
                loss.backward()
            if (step + 1) % GRADIENT_ACCUMULATION_STEPS == 0 or (step + 1) == len(train_loader):
                if USE_FP16:
                    scaler.step(optimizer)
                    scaler.update()
                else:
                    optimizer.step()
                optimizer.zero_grad()
        # Validation
        model.eval()
        val_preds, val_true = [], []
        with torch.no_grad():
            for batch in val_loader:
                inputs, labels = batch
                labels = labels.to(DEVICE)
                with autocast(device_type="cuda", enabled=USE_FP16):
                    outputs = model(
                        input_ids=inputs['input_ids'].to(DEVICE),
                        attention_mask=inputs['attention_mask'].to(DEVICE),
                        tabular=inputs['tabular'].to(DEVICE)
                    )
                    preds = torch.argmax(outputs, dim=1)
                    val_preds.append(preds.cpu().numpy())
                    val_true.append(labels.cpu().numpy())
        val_preds = np.concatenate(val_preds)
        val_true = np.concatenate(val_true)
        val_acc = accuracy_score(val_true, val_preds)
        val_f1_macro = f1_score(val_true, val_preds, average='macro')
        val_f1_per_class = f1_score(val_true, val_preds, average=None)

        # Early stopping logic
        if val_f1_macro > best_val_f1_macro:
            best_val_f1_macro = val_f1_macro
            best_val_acc = val_acc
            best_val_f1_per_class = val_f1_per_class
            # Save to CPU to save GPU mem
            best_state_dict = {k: v.cpu()
                               for k, v in model.state_dict().items()}
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(
                    f"Early stopping at epoch {epoch+1} (no val_f1_macro improvement in {patience} epochs)")
                break

    # Restore best weights before returning
    if best_state_dict is not None:
        model.load_state_dict(best_state_dict)
    return best_val_acc, best_val_f1_macro, best_val_f1_per_class

### funciones de preproceasado

In [ ]:

def procesar_tipos(ruta_phrases: str, num_labels=5):
    """
    Función generalizada para procesar archivos CSV multi-label con columnas de tipo 'label_1', ..., 'label_N'.
    """
    df = pd.read_csv(ruta_phrases)
    df['bias_type'] = df['bias_type'].astype(str)
    labeltags = []
    for n in range(1, num_labels + 1):
        col_name = f'label_{n}'
        labeltags.append(col_name)
        df[col_name] = df['bias_type'].apply(
            lambda x: str(n) in x.split(',')).astype(int)
    cols_to_drop = [c for c in [
        'idNew', 'idPhrase', 'bias_type'] if c in df.columns]
    df = df.drop(columns=cols_to_drop)
    df = df.fillna(0)
    return df, labeltags


def obtener_parejas_berta(ruta_news: str, ruta_phrases: str):
    df_news = pd.read_csv(ruta_news)
    df_phrases = pd.read_csv(ruta_phrases)
    df_news = df_news.drop(columns=['idNew', 'fecha', 'newspaper'])
    cols_a_borrar = [
        'count_X',
        'count_X_log',
        'ratio_words_X',
        'ratio_sentences_X'
    ]
    df_news = df_news.drop(columns=cols_a_borrar, errors='ignore')
    df_pbias = df_phrases.drop(columns=['idNew', 'idPhrase', 'bias_type'])
    df_pbias = df_pbias.rename(columns={'bias': 'label'})
    first_emb = df_pbias.columns.get_loc('emb_0')
    last_emb = df_pbias.columns.get_loc('emb_383')
    df_pbias = df_pbias.drop(df_pbias.columns[first_emb:last_emb+1], axis=1)
    first_emb = df_news.columns.get_loc('emb_0')
    last_emb = df_news.columns.get_loc('emb_383')
    df_news = df_news.drop(df_news.columns[first_emb:last_emb+1], axis=1)
    pairs = [(df_news, "news"), (df_pbias, "phrases")]
    return pairs

## SMOTETomek vs SMOTEEEN

### tipo de sesgo

In [ ]:
# =================== CONFIGURACIÓN ===================
MODEL_NAME = "PlanTL-GOB-ES/roberta-base-bne"
SEED = 42
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(SEED)
np.random.seed(SEED)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# ======= PARÁMETROS VARIABLES POR EXPERIMENTO ========
MAX_LEN = 128
BATCH_SIZE = 256
NUM_CLASSES = 2  # Cambia según tu tarea/clases
GRADIENT_ACCUMULATION_STEPS = 2
USE_FP16 = True

grid_search_params = {
    'lr':        [2e-5, 5e-5],
    'dropout':   [0.1, 0.3],
    'epochs':    [5],
}
param_grid = list(ParameterGrid(grid_search_params))
balanceadores = {
    "smoteen": SMOTEENN(random_state=SEED),
    "smotetomek": SMOTETomek(random_state=SEED)
}

# =================== PREPROCESADO DE LOS DATOS ===================

phrases_loc = './csv/processed_phrases_embed.csv'
df_ptype, labeltags = procesar_tipos(phrases_loc, num_labels=5)

first_emb = df_ptype.columns.get_loc('emb_0')
last_emb = df_ptype.columns.get_loc('emb_383')
df_noemb = df_ptype.drop(df_ptype.columns[first_emb:last_emb+1], axis=1)

X = df_noemb.drop(columns=labeltags)
tabular_cols = [col for col in X.columns if col != 'text']
text_col = 'text'

scaler = StandardScaler()
X_scaled = X.copy()
X_scaled[tabular_cols] = scaler.fit_transform(X_scaled[tabular_cols])
X_scaled[text_col] = X[text_col]

# =================== MAIN TRAINING LOOP ===================
for labeltag in labeltags:
    print(f"\n=== Grid search Roberta para {labeltag} ===")
    y = df_ptype[labeltag].values.astype(int)
    df_actual = X_scaled.copy()
    df_actual['label'] = y
    texts = df_actual[text_col].astype(str).tolist()
    tabular_data = df_actual[tabular_cols].values
    labels = y
    scaler = StandardScaler()
    tabular_scaled = scaler.fit_transform(tabular_data)
    dummy_tabular = np.zeros((len(df_actual), 1), dtype=np.float32)

    for use_tabular in [True, False]:
        feature_type = "all" if use_tabular else "embedding"
        print(
            f"\n>> Entrenando con: {'todos los atributos' if use_tabular else 'solo embeddings de RoBERTa'}")
        X_tab = tabular_scaled if use_tabular else dummy_tabular
        for balance_name, sampler in balanceadores.items():
            print(f"\n>> Balanceador: {balance_name}")
            skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=SEED)
            param_results = defaultdict(list)
            save_name = f"resultados_labeltags_{labeltag}_{balance_name}_{feature_type}.json"
            if os.path.exists(save_name):
                try:
                    with open(save_name, "r", encoding="utf-8") as f:
                        prev_res = json.load(f)
                    already_done = set(prev_res["results"].keys())
                    print(
                        f"[INFO] {save_name} encontrado, {len(already_done)}/{len(param_grid)} combinaciones ya guardadas.")
                except Exception as e:
                    print(f"[WARN] Error leyendo {save_name}: {e}")
                    already_done = set()
                    prev_res = {"params_grid": param_grid, "results": {}}
            else:
                already_done = set()
                prev_res = {"params_grid": param_grid, "results": {}}
            for params in tqdm(param_grid, desc=f"Grid {balance_name} {feature_type} {labeltag}"):
                params_key = str(params)
                if params_key in already_done:
                    continue  # Saltar combinaciones ya terminadas
                fold_accs, fold_f1_macros, fold_f1s_per_class = [], [], []
                for fold_idx, (train_idx, val_idx) in enumerate(skf.split(X_tab, labels)):
                    train_texts = [texts[idx] for idx in train_idx]
                    val_texts = [texts[idx] for idx in val_idx]
                    train_tokenized = tokenizer(
                        train_texts,
                        add_special_tokens=True,
                        truncation=True,
                        max_length=MAX_LEN,
                        padding='max_length',
                        return_tensors='pt'
                    )
                    val_tokenized = tokenizer(
                        val_texts,
                        add_special_tokens=True,
                        truncation=True,
                        max_length=MAX_LEN,
                        padding='max_length',
                        return_tensors='pt'
                    )
                    train_input_ids, val_input_ids = train_tokenized[
                        'input_ids'], val_tokenized['input_ids']
                    train_attention_mask, val_attention_mask = train_tokenized[
                        'attention_mask'], val_tokenized['attention_mask']
                    train_tabular, val_tabular = X_tab[train_idx], X_tab[val_idx]
                    train_labels, val_labels = labels[train_idx], labels[val_idx]

                    # ----------- BLOQUE ROBUSTO DE BALANCEO Y MAPEADO DE ÍNDICES ---------------
                    unique_classes, counts_classes = np.unique(
                        train_labels, return_counts=True)
                    print(
                        f"[INFO] Fold {fold_idx} clases: {dict(zip(unique_classes, counts_classes))}")
                    use_balanced = True
                    if len(unique_classes) < 2:
                        print(
                            f"[WARN] Fold {fold_idx} solo tiene una clase, no se puede balancear. Usando datos originales.")
                        tabular_sampl, labels_sampl = train_tabular, train_labels
                        idx_sampl = np.arange(len(train_tabular))
                        use_balanced = False
                    else:
                        min_samples = np.min(counts_classes)
                        k_neighbors = min(
                            5, min_samples-1) if min_samples > 1 else 1
                        if hasattr(sampler, "smote"):
                            sampler.set_params(
                                **{"smote__k_neighbors": k_neighbors})
                        try:
                            tabular_sampl, labels_sampl = sampler.fit_resample(
                                train_tabular, train_labels)
                            if tabular_sampl.shape[0] == 0:
                                print(
                                    f"[WARN] Balanceador dejó fold {fold_idx} vacío. Usando datos originales.")
                                tabular_sampl, labels_sampl = train_tabular, train_labels
                                idx_sampl = np.arange(len(train_tabular))
                                use_balanced = False
                            else:
                                from sklearn.neighbors import NearestNeighbors
                                nn_model = NearestNeighbors(n_neighbors=1)
                                nn_model.fit(train_tabular)
                                _, idx_sampl = nn_model.kneighbors(
                                    tabular_sampl)
                                idx_sampl = idx_sampl.flatten()
                        except Exception as e:
                            print(
                                f"[WARN] Error balanceando fold {fold_idx}: {e}. Usando datos originales.")
                            tabular_sampl, labels_sampl = train_tabular, train_labels
                            idx_sampl = np.arange(len(train_tabular))
                            use_balanced = False
                    # ---------------------------------------------------------------------------

                    train_input_ids_sampl = train_input_ids[idx_sampl]
                    train_attention_mask_sampl = train_attention_mask[idx_sampl]

                    # Construir datasets
                    train_dataset = TextTabularDataset(
                        train_input_ids_sampl, train_attention_mask_sampl, tabular_sampl, labels_sampl
                    )
                    val_dataset = TextTabularDataset(
                        val_input_ids, val_attention_mask, val_tabular, val_labels
                    )
                    train_loader = DataLoader(
                        train_dataset,
                        batch_size=BATCH_SIZE,
                        shuffle=True,
                        num_workers=4,
                        pin_memory=True
                    )
                    val_loader = DataLoader(
                        val_dataset,
                        batch_size=BATCH_SIZE,
                        shuffle=False,
                        num_workers=4,
                        pin_memory=True
                    )
                    present_classes = np.unique(labels_sampl)
                    weights_present = compute_class_weight(
                        class_weight='balanced',
                        classes=present_classes,
                        y=labels_sampl
                    )
                    full_weights = np.zeros(NUM_CLASSES, dtype=np.float32)
                    for cls, w in zip(present_classes, weights_present):
                        full_weights[int(cls)] = w
                    fold_class_weights = torch.tensor(
                        full_weights, dtype=torch.float32, device=DEVICE)
                    model = RobertaWithTabularDeepHead(
                        tabular_dim=train_tabular.shape[1],
                        num_classes=NUM_CLASSES,
                        dropout=params['dropout']
                    )
                    val_acc, val_f1_macro, val_f1_per_class = train_and_validate(
                        model,
                        train_loader,
                        val_loader,
                        class_weights=fold_class_weights,
                        epochs=params['epochs'],
                        lr=params['lr']
                    )
                    fold_accs.append(val_acc)
                    fold_f1_macros.append(val_f1_macro)
                    fold_f1s_per_class.append(val_f1_per_class)
                if len(fold_accs) == 0:
                    print(
                        f"[WARN] Ningún fold válido para {params} en {labeltag} con {balance_name} y {'all' if use_tabular else 'embedding'}")
                    continue
                mean_acc = float(np.mean(fold_accs))
                mean_f1_macro = float(np.mean(fold_f1_macros))
                std_acc = float(np.std(fold_accs))
                mean_f1_per_class = np.mean(fold_f1s_per_class, axis=0)
                mean_f1_per_class_dict = {f'clase_{cl}': float(f1) for cl, f1 in zip(
                    sorted(np.unique(labels)), mean_f1_per_class)}
                result = {
                    'params': params,
                    'mean_acc': mean_acc,
                    'mean_f1_macro': mean_f1_macro,
                    'std_acc': std_acc,
                    'mean_f1_per_class': mean_f1_per_class_dict,
                    'fold_results_acc': [float(x) for x in fold_accs],
                    'fold_results_f1_macro': [float(x) for x in fold_f1_macros],
                    'fold_results_f1_per_class': [f1.tolist() for f1 in fold_f1s_per_class]
                }
                prev_res["results"][params_key] = result
                # Guardado incremental tras cada combinación
                with open(save_name, "w", encoding="utf-8") as f:
                    json.dump(prev_res, f, indent=2, ensure_ascii=False)
            print(f"[OK] Resultados guardados en {save_name}")

### Orientación de sesgo

In [ ]:
# =================== CONFIGURACIÓN ===================
MODEL_NAME = "PlanTL-GOB-ES/roberta-base-bne"
SEED = 42
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_CLASSES = 3

torch.manual_seed(SEED)
np.random.seed(SEED)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

GRADIENT_ACCUMULATION_STEPS = 2
USE_FP16 = True

news_loc = './csv/news.csv'
phrases_loc = './csv/processed_phrases_embed.csv'

pairs = obtener_parejas_berta(news_loc, phrases_loc)

grid_search_params = {
    'lr':        [2e-5, 5e-5],
    'dropout':   [0.1, 0.3],
    'epochs':    [5],
}
param_grid = list(ParameterGrid(grid_search_params))
balanceadores = {
    "smoteen": SMOTEENN(random_state=SEED),
    "smotetomek": SMOTETomek(random_state=SEED)
}

BATCH_SIZE_CONFIG = {
    "news": 32,
    "phrases": 256
}
MAXLEN_CONFIG = {
    "news": 512,
    "phrases": 128
}

# =================== MAIN TRAINING LOOP ===================
# SOLO _all.json: solo atributos tabulares+embeddings (NO solo embeddings)
for i, (df, dataset_name) in enumerate(pairs):
    print(f"\n=========== Processing: {dataset_name} ===========")
    MAX_LEN = MAXLEN_CONFIG[dataset_name]
    BATCH_SIZE = BATCH_SIZE_CONFIG[dataset_name]
    df = df.fillna(0).reset_index(drop=True)

    def label_cat(x):
        if x > 0.5:
            return 1
        elif x < -0.5:
            return 2
        else:
            return 0
    df['label_cat'] = df['label'].apply(label_cat)
    texts = df['text'].astype(str).tolist()
    tabular_cols = [col for col in df.columns if col not in [
        'label', 'text', 'label_cat']]
    tabular_data = df[tabular_cols].values
    labels = df['label_cat'].values.astype(int)
    scaler = StandardScaler()
    tabular_scaled = scaler.fit_transform(tabular_data)

    # SOLO el caso "all" (con atributos tabulares)
    feature_type = "all"
    print(f"\n>> Entrenando con: todos los atributos (tabulares + embeddings de RoBERTa)")
    X_tab = tabular_scaled
    for balance_name, sampler in balanceadores.items():
        print(f"\n>> Balanceador: {balance_name}")
        kfold = KFold(n_splits=3, shuffle=True, random_state=SEED)
        save_name = f"resultados_{dataset_name}_{balance_name}_{feature_type}.json"
        # --- Cargar resultados previos si existen ---
        if os.path.exists(save_name):
            try:
                with open(save_name, "r", encoding="utf-8") as f:
                    prev_res = json.load(f)
                already_done = set(prev_res["results"].keys())
                print(
                    f"[INFO] {save_name} encontrado, {len(already_done)}/{len(param_grid)} combinaciones ya guardadas.")
            except Exception as e:
                print(f"[WARN] Error leyendo {save_name}: {e}")
                already_done = set()
                prev_res = {"params_grid": param_grid, "results": {}}
        else:
            already_done = set()
            prev_res = {"params_grid": param_grid, "results": {}}
        for params in tqdm(param_grid, desc=f"Grid {balance_name} {feature_type}"):
            params_key = str(params)
            if params_key in already_done:
                continue  # Saltar combinaciones ya terminadas
            fold_accs, fold_f1_macros, fold_f1s_per_class = [], [], []
            for fold_idx, (train_idx, val_idx) in enumerate(kfold.split(X_tab, labels)):
                # Tokenización por split para ahorrar RAM
                train_texts = [texts[idx] for idx in train_idx]
                val_texts = [texts[idx] for idx in val_idx]
                train_tokenized = tokenizer(
                    train_texts,
                    add_special_tokens=True,
                    truncation=True,
                    max_length=MAX_LEN,
                    padding='max_length',
                    return_tensors='pt'
                )
                val_tokenized = tokenizer(
                    val_texts,
                    add_special_tokens=True,
                    truncation=True,
                    max_length=MAX_LEN,
                    padding='max_length',
                    return_tensors='pt'
                )
                train_input_ids, val_input_ids = train_tokenized['input_ids'], val_tokenized['input_ids']
                train_attention_mask, val_attention_mask = train_tokenized[
                    'attention_mask'], val_tokenized['attention_mask']
                train_tabular, val_tabular = X_tab[train_idx], X_tab[val_idx]
                train_labels, val_labels = labels[train_idx], labels[val_idx]

                # === BLOQUE ROBUSTO DE BALANCEO ===
                if np.all(train_tabular == train_tabular[0]):
                    print(
                        f"[WARN] Fold {fold_idx}: solo embeddings (dummy tabular). No se balancea.")
                    tabular_sampl, labels_sampl = train_tabular, train_labels
                    idx_sampl = np.arange(len(train_tabular))
                elif len(np.unique(train_labels)) < 2:
                    print(
                        f"[WARN] Fold {fold_idx}: solo hay una clase, no se puede balancear.")
                    tabular_sampl, labels_sampl = train_tabular, train_labels
                    idx_sampl = np.arange(len(train_tabular))
                else:
                    # Ajusta k_neighbors para SMOTE si hay pocos positivos
                    bincounts = np.bincount(train_labels)
                    min_samples = np.min(bincounts[bincounts > 0])
                    k_neighbors = min(
                        5, min_samples-1) if min_samples > 1 else 1
                    if hasattr(sampler, "smote") and getattr(sampler, "smote", None) is not None:
                        sampler.set_params(
                            **{"smote__k_neighbors": k_neighbors})
                    try:
                        tabular_sampl, labels_sampl = sampler.fit_resample(
                            train_tabular, train_labels)
                        if tabular_sampl.shape[0] == 0:
                            print(
                                f"[WARN] Fold {fold_idx}: el balanceador devolvió 0 muestras. No se balancea.")
                            tabular_sampl, labels_sampl = train_tabular, train_labels
                            idx_sampl = np.arange(len(train_tabular))
                        else:
                            from sklearn.neighbors import NearestNeighbors
                            nn_model = NearestNeighbors(n_neighbors=1)
                            nn_model.fit(train_tabular)
                            _, idx_sampl = nn_model.kneighbors(tabular_sampl)
                            idx_sampl = idx_sampl.flatten()
                    except Exception as e:
                        warnings.warn(
                            f"Fold {fold_idx}: error en balanceador: {e}. No se balancea.")
                        tabular_sampl, labels_sampl = train_tabular, train_labels
                        idx_sampl = np.arange(len(train_tabular))

                train_input_ids_sampl = train_input_ids[idx_sampl]
                train_attention_mask_sampl = train_attention_mask[idx_sampl]
                train_dataset = TextTabularDataset(
                    train_input_ids_sampl, train_attention_mask_sampl, tabular_sampl, labels_sampl
                )
                val_dataset = TextTabularDataset(
                    val_input_ids, val_attention_mask, val_tabular, val_labels
                )
                train_loader = DataLoader(
                    train_dataset,
                    batch_size=BATCH_SIZE,
                    shuffle=True,
                    num_workers=4,
                    pin_memory=True
                )
                val_loader = DataLoader(
                    val_dataset,
                    batch_size=BATCH_SIZE,
                    shuffle=False,
                    num_workers=4,
                    pin_memory=True
                )
                present_classes = np.unique(labels_sampl)
                weights_present = compute_class_weight(
                    class_weight='balanced',
                    classes=present_classes,
                    y=labels_sampl
                )
                full_weights = np.zeros(NUM_CLASSES, dtype=np.float32)
                for cls, w in zip(present_classes, weights_present):
                    full_weights[int(cls)] = w
                fold_class_weights = torch.tensor(
                    full_weights, dtype=torch.float32, device=DEVICE)
                model = RobertaWithTabularDeepHead(
                    tabular_dim=train_tabular.shape[1],
                    num_classes=NUM_CLASSES,
                    dropout=params['dropout']
                )
                val_acc, val_f1_macro, val_f1_per_class = train_and_validate(
                    model,
                    train_loader,
                    val_loader,
                    class_weights=fold_class_weights,
                    epochs=params['epochs'],
                    lr=params['lr']
                )
                fold_accs.append(val_acc)
                fold_f1_macros.append(val_f1_macro)
                fold_f1s_per_class.append(val_f1_per_class)
            mean_acc = float(np.mean(fold_accs))
            mean_f1_macro = float(np.mean(fold_f1_macros))
            std_acc = float(np.std(fold_accs))
            mean_f1_per_class = np.mean(fold_f1s_per_class, axis=0)
            mean_f1_per_class_dict = {f'clase_{cl}': float(f1) for cl, f1 in zip(
                sorted(np.unique(labels)), mean_f1_per_class)}
            result = {
                'params': params,
                'mean_acc': mean_acc,
                'mean_f1_macro': mean_f1_macro,
                'std_acc': std_acc,
                'mean_f1_per_class': mean_f1_per_class_dict,
                'fold_results_acc': [float(x) for x in fold_accs],
                'fold_results_f1_macro': [float(x) for x in fold_f1_macros],
                'fold_results_f1_per_class': [f1.tolist() for f1 in fold_f1s_per_class]
            }
            prev_res["results"][params_key] = result
            # Guardado incremental tras cada combinación
            with open(save_name, "w", encoding="utf-8") as f:
                json.dump(prev_res, f, indent=2, ensure_ascii=False)
        print(f"[OK] Resultados guardados en {save_name}")

## Balanceadores mixtos simples (Embeddings vs Embeddings + Tabular)

In [ ]:

def balanceo_mixto_multi(tabular, labels, input_ids, att_mask, maj_perc=0.7, min_perc=0.6, random_state=42):
    """
    Oversample y undersample tabular, input_ids, att_mask y labels a la vez.
    Devuelve: tabular_res, input_ids_res, att_mask_res, y_res
    """
    classes, counts = np.unique(labels, return_counts=True)
    if len(classes) != 2:
        raise ValueError("Solo soportado para binario")
    maj = classes[np.argmax(counts)]
    min_ = classes[np.argmin(counts)]
    n_maj = counts[np.argmax(counts)]
    target_maj = int(n_maj * maj_perc)
    target_min = int(n_maj * min_perc)
    # Concatenar features para que todas sean oversampleadas igual
    X_all = np.concatenate(
        [tabular, input_ids, att_mask], axis=1
    )
    ros = RandomOverSampler(
        sampling_strategy={maj: n_maj, min_: target_min}, random_state=random_state)
    rus = RandomUnderSampler(sampling_strategy={
                             maj: target_maj, min_: target_min}, random_state=random_state)
    X_res, y_res = ros.fit_resample(X_all, labels)
    X_res, y_res = rus.fit_resample(X_res, y_res)
    # Separar de vuelta
    n_tab = tabular.shape[1]
    n_ids = input_ids.shape[1]
    n_mask = att_mask.shape[1]
    tabular_res = X_res[:, :n_tab]
    input_ids_res = X_res[:, n_tab:n_tab+n_ids].astype(int)
    att_mask_res = X_res[:, n_tab+n_ids:].astype(int)
    return tabular_res, input_ids_res, att_mask_res, y_res

### Tipo de sesgo

In [ ]:
# =================== CONFIGURACIÓN ===================
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
MODEL_NAME = "PlanTL-GOB-ES/roberta-base-bne"
SEED = 42
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_CLASSES = 2  # Cambia a tu número de clases

torch.manual_seed(SEED)
np.random.seed(SEED)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# =============== CARGA Y PREPROCESADO ===============
phrases_loc = './csv/processed_phrases_embed.csv'

df_ptype, labeltags = procesar_tipos(phrases_loc)

first_emb = df_ptype.columns.get_loc('emb_0')
last_emb = df_ptype.columns.get_loc('emb_383')
df_noemb = df_ptype.drop(df_ptype.columns[first_emb:last_emb+1], axis=1)

X = df_noemb.drop(columns=labeltags)
tabular_cols = [col for col in X.columns if col != 'text']
text_col = 'text'

scaler = StandardScaler()
X_scaled = X.copy()
X_scaled[tabular_cols] = scaler.fit_transform(X_scaled[tabular_cols])
X_scaled[text_col] = X[text_col]

# PARAMS Y BALANCEO
grid_search_params = {
    'lr':        [2e-5, 5e-5],
    'dropout':   [0.1, 0.3],
    'epochs':    [5]
}
param_grid = list(ParameterGrid(grid_search_params))

MAX_LEN = 128
BATCH_SIZE = 256

# MAIN TRAINING LOOP
for labeltag in labeltags:
    print(f"\n=== Grid search Roberta para {labeltag} ===")
    y = df_ptype[labeltag].values.astype(int)
    df_actual = X_scaled.copy()
    df_actual['label'] = y
    texts = df_actual[text_col].astype(str).tolist()
    tabular_data = df_actual[tabular_cols].values
    labels = y
    scaler = StandardScaler()
    tabular_scaled = scaler.fit_transform(tabular_data)
    dummy_tabular = np.zeros((len(df_actual), 1), dtype=np.float32)

    for use_tabular in [True, False]:
        feature_type = "all" if use_tabular else "embedding"
        print(
            f"\n>> Entrenando con: {'todos los atributos' if use_tabular else 'solo embeddings de RoBERTa'}")
        X_tab = tabular_scaled if use_tabular else dummy_tabular
        skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=SEED)
        param_results = defaultdict(list)
        save_name = f"comparativa_{labeltag}_{feature_type}_balanceo_mixto.json"
        if os.path.exists(save_name):
            try:
                with open(save_name, "r", encoding="utf-8") as f:
                    prev_res = json.load(f)
                already_done = set(prev_res["results"].keys())
                print(
                    f"[INFO] {save_name} encontrado, {len(already_done)}/{len(param_grid)} combinaciones ya guardadas.")
            except Exception as e:
                print(f"[WARN] Error leyendo {save_name}: {e}")
                already_done = set()
                prev_res = {"params_grid": param_grid, "results": {}}
        else:
            already_done = set()
            prev_res = {"params_grid": param_grid, "results": {}}
        for params in tqdm(param_grid, desc=f"Grid {feature_type} {labeltag}"):
            params_key = str(params)
            if params_key in already_done:
                continue
            fold_accs, fold_f1_macros, fold_f1s_per_class = [], [], []
            for fold_idx, (train_idx, val_idx) in enumerate(skf.split(X_tab, labels)):
                train_texts = [texts[idx] for idx in train_idx]
                val_texts = [texts[idx] for idx in val_idx]
                train_tokenized = tokenizer(
                    train_texts,
                    add_special_tokens=True,
                    truncation=True,
                    max_length=MAX_LEN,
                    padding='max_length',
                    return_tensors='np'
                )
                val_tokenized = tokenizer(
                    val_texts,
                    add_special_tokens=True,
                    truncation=True,
                    max_length=MAX_LEN,
                    padding='max_length',
                    return_tensors='pt'
                )
                train_input_ids = train_tokenized['input_ids']
                train_attention_mask = train_tokenized['attention_mask']
                val_input_ids = val_tokenized['input_ids']
                val_attention_mask = val_tokenized['attention_mask']
                train_tabular, val_tabular = X_tab[train_idx], X_tab[val_idx]
                train_labels, val_labels = labels[train_idx], labels[val_idx]

                # BALANCEO MIXTO en ambos escenarios (MULTIARRAY)
                tabular_sampl, input_ids_sampl, att_mask_sampl, labels_sampl = balanceo_mixto_multi(
                    train_tabular, train_labels, train_input_ids, train_attention_mask,
                    maj_perc=0.7, min_perc=0.6, random_state=SEED
                )
                train_input_ids_sampl = torch.tensor(
                    input_ids_sampl, dtype=torch.long)
                train_attention_mask_sampl = torch.tensor(
                    att_mask_sampl, dtype=torch.long)
                tabular_sampl = tabular_sampl.astype(np.float32)

                # Construir datasets
                train_dataset = TextTabularDataset(
                    train_input_ids_sampl, train_attention_mask_sampl, tabular_sampl, labels_sampl
                )
                val_dataset = TextTabularDataset(
                    val_input_ids, val_attention_mask, val_tabular, val_labels
                )
                train_loader = DataLoader(
                    train_dataset,
                    batch_size=BATCH_SIZE,
                    shuffle=True,
                    num_workers=4,
                    pin_memory=True
                )
                val_loader = DataLoader(
                    val_dataset,
                    batch_size=BATCH_SIZE,
                    shuffle=False,
                    num_workers=4,
                    pin_memory=True
                )
                present_classes = np.unique(labels_sampl)
                weights_present = compute_class_weight(
                    class_weight='balanced',
                    classes=present_classes,
                    y=labels_sampl
                )
                full_weights = np.zeros(NUM_CLASSES, dtype=np.float32)
                for cls, w in zip(present_classes, weights_present):
                    full_weights[int(cls)] = w
                fold_class_weights = torch.tensor(
                    full_weights, dtype=torch.float32, device=DEVICE)
                model = RobertaWithTabularDeepHead(
                    tabular_dim=train_tabular.shape[1],
                    num_classes=NUM_CLASSES,
                    dropout=params['dropout']
                )
                val_acc, val_f1_macro, val_f1_per_class = train_and_validate(
                    model,
                    train_loader,
                    val_loader,
                    class_weights=fold_class_weights,
                    epochs=params['epochs'],
                    lr=params['lr']
                )
                fold_accs.append(val_acc)
                fold_f1_macros.append(val_f1_macro)
                fold_f1s_per_class.append(val_f1_per_class)
            if len(fold_accs) == 0:
                print(
                    f"[WARN] Ningún fold válido para {params} en {labeltag} con {feature_type}")
                continue
            mean_acc = float(np.mean(fold_accs))
            mean_f1_macro = float(np.mean(fold_f1_macros))
            std_acc = float(np.std(fold_accs))
            mean_f1_per_class = np.mean(fold_f1s_per_class, axis=0)
            mean_f1_per_class_dict = {f'clase_{cl}': float(f1) for cl, f1 in zip(
                sorted(np.unique(labels)), mean_f1_per_class)}
            result = {
                'params': params,
                'mean_acc': mean_acc,
                'mean_f1_macro': mean_f1_macro,
                'std_acc': std_acc,
                'mean_f1_per_class': mean_f1_per_class_dict,
                'fold_results_acc': [float(x) for x in fold_accs],
                'fold_results_f1_macro': [float(x) for x in fold_f1_macros],
                'fold_results_f1_per_class': [f1.tolist() for f1 in fold_f1s_per_class]
            }
            prev_res["results"][params_key] = result
            # Guardado incremental tras cada combinación
            with open(save_name, "w", encoding="utf-8") as f:
                json.dump(prev_res, f, indent=2, ensure_ascii=False)
        print(f"[OK] Resultados guardados en {save_name}")

### Orientación del sesgo

In [ ]:
MODEL_NAME = "PlanTL-GOB-ES/roberta-base-bne"
SEED = 42
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_CLASSES = 3

torch.manual_seed(SEED)
np.random.seed(SEED)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

news_loc = './csv/news.csv'
phrases_loc = './csv/processed_phrases_embed.csv'

pairs = obtener_parejas_berta(news_loc, phrases_loc)

grid_search_params = {
    'lr':        [2e-5, 5e-5],
    'dropout':   [0.1, 0.3],
    'epochs':    [5],
}
param_grid = list(ParameterGrid(grid_search_params))

BATCH_SIZE_CONFIG = {
    "news": 32,
    "phrases": 256
}
MAXLEN_CONFIG = {
    "news": 512,
    "phrases": 128
}


def balanceo_mixto_multi(tabular, labels, input_ids, att_mask, maj_perc=0.7, min_perc=0.6, random_state=42):
    classes, counts = np.unique(labels, return_counts=True)
    if len(classes) < 2:
        raise ValueError("Se requiere al menos dos clases")
    maj = classes[np.argmax(counts)]
    min_ = classes[np.argmin(counts)]
    n_maj = counts[np.argmax(counts)]
    target_maj = int(n_maj * maj_perc)
    target_min = int(n_maj * min_perc)
    X_all = np.concatenate([tabular, input_ids, att_mask], axis=1)
    ros = RandomOverSampler(
        sampling_strategy={maj: n_maj, min_: target_min}, random_state=random_state)
    rus = RandomUnderSampler(sampling_strategy={
                             maj: target_maj, min_: target_min}, random_state=random_state)
    X_res, y_res = ros.fit_resample(X_all, labels)
    X_res, y_res = rus.fit_resample(X_res, y_res)
    n_tab = tabular.shape[1]
    n_ids = input_ids.shape[1]
    n_mask = att_mask.shape[1]
    tabular_res = X_res[:, :n_tab]
    input_ids_res = X_res[:, n_tab:n_tab+n_ids].astype(int)
    att_mask_res = X_res[:, n_tab+n_ids:].astype(int)
    return tabular_res, input_ids_res, att_mask_res, y_res


for i, (df, dataset_name) in enumerate(pairs):
    print(f"\n=========== Processing: {dataset_name} ===========")
    MAX_LEN = MAXLEN_CONFIG[dataset_name]
    BATCH_SIZE = BATCH_SIZE_CONFIG[dataset_name]
    df = df.fillna(0).reset_index(drop=True)

    def label_cat(x):
        if x > 0.5:
            return 1
        elif x < -0.5:
            return 2
        else:
            return 0
    df['label_cat'] = df['label'].apply(label_cat)
    texts = df['text'].astype(str).tolist()
    tabular_cols = [col for col in df.columns if col not in [
        'label', 'text', 'label_cat']]
    tabular_data = df[tabular_cols].values
    labels = df['label_cat'].values.astype(int)
    scaler = StandardScaler()
    tabular_scaled = scaler.fit_transform(tabular_data)
    dummy_tabular = np.zeros((len(df), 1), dtype=np.float32)

    for use_tabular in [True, False]:
        feature_type = "all" if use_tabular else "embedding"
        print(
            f"\n>> Entrenando con: {'todos los atributos' if use_tabular else 'solo embeddings de RoBERTa'}")
        X_tab = tabular_scaled if use_tabular else dummy_tabular
        kfold = KFold(n_splits=3, shuffle=True, random_state=SEED)
        save_name = f"resultados_{dataset_name}_balanceo_mixto_{feature_type}.json"
        if os.path.exists(save_name):
            try:
                with open(save_name, "r", encoding="utf-8") as f:
                    prev_res = json.load(f)
                already_done = set(prev_res["results"].keys())
                print(
                    f"[INFO] {save_name} encontrado, {len(already_done)}/{len(param_grid)} combinaciones ya guardadas.")
            except Exception as e:
                print(f"[WARN] Error leyendo {save_name}: {e}")
                already_done = set()
                prev_res = {"params_grid": param_grid, "results": {}}
        else:
            already_done = set()
            prev_res = {"params_grid": param_grid, "results": {}}
        for params in tqdm(param_grid, desc=f"Grid balanceo_mixto {feature_type}"):
            params_key = str(params)
            if params_key in already_done:
                continue
            fold_accs, fold_f1_macros, fold_f1s_per_class = [], [], []
            for fold_idx, (train_idx, val_idx) in enumerate(kfold.split(X_tab, labels)):
                train_texts = [texts[idx] for idx in train_idx]
                val_texts = [texts[idx] for idx in val_idx]
                train_tokenized = tokenizer(
                    train_texts,
                    add_special_tokens=True,
                    truncation=True,
                    max_length=MAX_LEN,
                    padding='max_length',
                    return_tensors='np'
                )
                val_tokenized = tokenizer(
                    val_texts,
                    add_special_tokens=True,
                    truncation=True,
                    max_length=MAX_LEN,
                    padding='max_length',
                    return_tensors='pt'
                )
                train_input_ids = train_tokenized['input_ids']
                train_attention_mask = train_tokenized['attention_mask']
                val_input_ids = val_tokenized['input_ids']
                val_attention_mask = val_tokenized['attention_mask']
                train_tabular, val_tabular = X_tab[train_idx], X_tab[val_idx]
                train_labels, val_labels = labels[train_idx], labels[val_idx]

                tabular_sampl, input_ids_sampl, att_mask_sampl, labels_sampl = balanceo_mixto_multi(
                    train_tabular, train_labels, train_input_ids, train_attention_mask,
                    maj_perc=0.7, min_perc=0.6, random_state=SEED
                )
                train_input_ids_sampl = torch.tensor(
                    input_ids_sampl, dtype=torch.long)
                train_attention_mask_sampl = torch.tensor(
                    att_mask_sampl, dtype=torch.long)
                tabular_sampl = tabular_sampl.astype(np.float32)

                train_dataset = TextTabularDataset(
                    train_input_ids_sampl, train_attention_mask_sampl, tabular_sampl, labels_sampl
                )
                val_dataset = TextTabularDataset(
                    val_input_ids, val_attention_mask, val_tabular, val_labels
                )
                train_loader = DataLoader(
                    train_dataset,
                    batch_size=BATCH_SIZE,
                    shuffle=True,
                    num_workers=4,
                    pin_memory=True
                )
                val_loader = DataLoader(
                    val_dataset,
                    batch_size=BATCH_SIZE,
                    shuffle=False,
                    num_workers=4,
                    pin_memory=True
                )
                present_classes = np.unique(labels_sampl)
                weights_present = compute_class_weight(
                    class_weight='balanced',
                    classes=present_classes,
                    y=labels_sampl
                )
                full_weights = np.zeros(NUM_CLASSES, dtype=np.float32)
                for cls, w in zip(present_classes, weights_present):
                    full_weights[int(cls)] = w
                fold_class_weights = torch.tensor(
                    full_weights, dtype=torch.float32, device=DEVICE)
                model = RobertaWithTabularDeepHead(
                    tabular_dim=train_tabular.shape[1],
                    num_classes=NUM_CLASSES,
                    dropout=params['dropout']
                )
                val_acc, val_f1_macro, val_f1_per_class = train_and_validate(
                    model,
                    train_loader,
                    val_loader,
                    class_weights=fold_class_weights,
                    epochs=params['epochs'],
                    lr=params['lr']
                )
                fold_accs.append(val_acc)
                fold_f1_macros.append(val_f1_macro)
                fold_f1s_per_class.append(val_f1_per_class)
            mean_acc = float(np.mean(fold_accs))
            mean_f1_macro = float(np.mean(fold_f1_macros))
            std_acc = float(np.std(fold_accs))
            mean_f1_per_class = np.mean(fold_f1s_per_class, axis=0)
            mean_f1_per_class_dict = {f'clase_{cl}': float(f1) for cl, f1 in zip(
                sorted(np.unique(labels)), mean_f1_per_class)}
            result = {
                'params': params,
                'mean_acc': mean_acc,
                'mean_f1_macro': mean_f1_macro,
                'std_acc': std_acc,
                'mean_f1_per_class': mean_f1_per_class_dict,
                'fold_results_acc': [float(x) for x in fold_accs],
                'fold_results_f1_macro': [float(x) for x in fold_f1_macros],
                'fold_results_f1_per_class': [f1.tolist() for f1 in fold_f1s_per_class]
            }
            prev_res["results"][params_key] = result
            with open(save_name, "w", encoding="utf-8") as f:
                json.dump(prev_res, f, indent=2, ensure_ascii=False)
        print(f"[OK] Resultados guardados en {save_name}")

## Undersampling (Embeddings vs Embeddings + Tabular)

In [ ]:
def undersample_multi(tabular, labels, input_ids, att_mask, random_state=42):
    classes, counts = np.unique(labels, return_counts=True)
    if len(classes) < 2:
        raise ValueError("Se requiere al menos dos clases")
    min_count = np.min(counts)
    sampling_strategy = {cls: min_count for cls in classes}
    X_all = np.concatenate([tabular, input_ids, att_mask], axis=1)
    rus = RandomUnderSampler(
        sampling_strategy=sampling_strategy, random_state=random_state)
    X_res, y_res = rus.fit_resample(X_all, labels)
    n_tab = tabular.shape[1]
    n_ids = input_ids.shape[1]
    n_mask = att_mask.shape[1]
    tabular_res = X_res[:, :n_tab]
    input_ids_res = X_res[:, n_tab:n_tab+n_ids].astype(int)
    att_mask_res = X_res[:, n_tab+n_ids:].astype(int)
    return tabular_res, input_ids_res, att_mask_res, y_res

### Tipo de sesgo

In [ ]:
MODEL_NAME = "PlanTL-GOB-ES/roberta-base-bne"
SEED = 42
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_CLASSES = 2

torch.manual_seed(SEED)
np.random.seed(SEED)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

phrases_loc = './csv/processed_phrases_embed.csv'


def procesar_tipos(ruta_phrases: str):
    df_ptype = pd.read_csv(ruta_phrases)
    df_ptype['bias_type'] = df_ptype['bias_type'].astype(str)
    labeltags = []
    for n in range(1, 6):
        col_name = f'label_{n}'
        labeltags.append(col_name)
        df_ptype[col_name] = df_ptype['bias_type'].apply(
            lambda x: str(n) in x.split(',')).astype(int)
    df_ptype = df_ptype.drop(columns=['idNew', 'idPhrase', 'bias_type'])
    df_ptype = df_ptype.fillna(0)
    return df_ptype, labeltags


df_ptype, labeltags = procesar_tipos(phrases_loc)
first_emb = df_ptype.columns.get_loc('emb_0')
last_emb = df_ptype.columns.get_loc('emb_383')
embedding_cols = [col for col in df_ptype.columns[first_emb:last_emb+1]]

grid_search_params = {
    'lr':        [2e-5, 5e-5],
    'dropout':   [0.1, 0.3],
    'epochs':    [5],
}
param_grid = list(ParameterGrid(grid_search_params))

MAX_LEN = 128
BATCH_SIZE = 256
GRADIENT_ACCUMULATION_STEPS = 2
USE_FP16 = True

for labeltag in labeltags:
    print(f"\n=== Grid search Roberta para {labeltag} ===")
    y = df_ptype[labeltag].values.astype(int)
    texts = df_ptype['text'].astype(str).tolist()
    labels = y

    X_tab_all = df_ptype.drop(
        columns=labeltags + ['text'] + embedding_cols).values
    scaler_all = StandardScaler()
    X_tab_all_scaled = scaler_all.fit_transform(X_tab_all)
    feature_types = [
        ("all", X_tab_all_scaled),
        ("embedding", df_ptype[embedding_cols].values)
    ]
    for feature_type, X_tab in feature_types:
        print(
            f"\n>> Entrenando con: {'todas las features' if feature_type == 'all' else 'solo embeddings'}")
        skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=SEED)
        param_results = defaultdict(list)
        save_name = f"comparativa_{labeltag}_{feature_type}_undersampling.json"
        already_done = set()
        if os.path.exists(save_name):
            try:
                with open(save_name, "r", encoding="utf-8") as f:
                    prev_res = json.load(f)
                already_done = set(prev_res["results"].keys())
                print(
                    f"[INFO] {save_name} encontrado, {len(already_done)}/{len(param_grid)} combinaciones ya guardadas.")
            except Exception as e:
                print(f"[WARN] Error leyendo {save_name}: {e}")
                prev_res = {"params_grid": param_grid, "results": {}}
        else:
            prev_res = {"params_grid": param_grid, "results": {}}
        for params in tqdm(param_grid, desc=f"Grid undersampling {feature_type} {labeltag}"):
            params_key = str(params)
            if params_key in already_done:
                continue
            fold_accs, fold_f1_macros, fold_f1s_per_class = [], [], []
            for fold_idx, (train_idx, val_idx) in enumerate(skf.split(X_tab, labels)):
                train_texts = [texts[idx] for idx in train_idx]
                val_texts = [texts[idx] for idx in val_idx]
                train_tokenized = tokenizer(
                    train_texts,
                    add_special_tokens=True,
                    truncation=True,
                    max_length=MAX_LEN,
                    padding='max_length',
                    return_tensors='np'
                )
                val_tokenized = tokenizer(
                    val_texts,
                    add_special_tokens=True,
                    truncation=True,
                    max_length=MAX_LEN,
                    padding='max_length',
                    return_tensors='pt'
                )
                train_input_ids = train_tokenized['input_ids']
                train_attention_mask = train_tokenized['attention_mask']
                val_input_ids = val_tokenized['input_ids']
                val_attention_mask = val_tokenized['attention_mask']
                train_tabular, val_tabular = X_tab[train_idx], X_tab[val_idx]
                train_labels, val_labels = labels[train_idx], labels[val_idx]

                tabular_sampl, input_ids_sampl, att_mask_sampl, labels_sampl = undersample_multi(
                    train_tabular, train_labels,
                    train_input_ids, train_attention_mask,
                    random_state=SEED
                )
                train_input_ids_sampl = torch.tensor(
                    input_ids_sampl, dtype=torch.long)
                train_attention_mask_sampl = torch.tensor(
                    att_mask_sampl, dtype=torch.long)
                tabular_sampl = tabular_sampl.astype(np.float32)

                train_dataset = TextTabularDataset(
                    train_input_ids_sampl, train_attention_mask_sampl, tabular_sampl, labels_sampl
                )
                val_dataset = TextTabularDataset(
                    val_input_ids, val_attention_mask, val_tabular, val_labels
                )
                train_loader = DataLoader(
                    train_dataset,
                    batch_size=BATCH_SIZE,
                    shuffle=True,
                    num_workers=4,
                    pin_memory=True
                )
                val_loader = DataLoader(
                    val_dataset,
                    batch_size=BATCH_SIZE,
                    shuffle=False,
                    num_workers=4,
                    pin_memory=True
                )
                present_classes = np.unique(labels_sampl)
                weights_present = compute_class_weight(
                    class_weight='balanced',
                    classes=present_classes,
                    y=labels_sampl
                )
                full_weights = np.zeros(NUM_CLASSES, dtype=np.float32)
                for cls, w in zip(present_classes, weights_present):
                    full_weights[int(cls)] = w
                fold_class_weights = torch.tensor(
                    full_weights, dtype=torch.float32, device=DEVICE)
                model = RobertaWithTabularDeepHead(
                    tabular_dim=train_tabular.shape[1],
                    num_classes=NUM_CLASSES,
                    dropout=params['dropout']
                )
                val_acc, val_f1_macro, val_f1_per_class = train_and_validate(
                    model,
                    train_loader,
                    val_loader,
                    class_weights=fold_class_weights,
                    epochs=params['epochs'],
                    lr=params['lr']
                )
                fold_accs.append(val_acc)
                fold_f1_macros.append(val_f1_macro)
                fold_f1s_per_class.append(val_f1_per_class)
            if len(fold_accs) == 0:
                print(
                    f"[WARN] Ningún fold válido para {params} en {labeltag} con {feature_type}")
                continue
            mean_acc = float(np.mean(fold_accs))
            mean_f1_macro = float(np.mean(fold_f1_macros))
            std_acc = float(np.std(fold_accs))
            mean_f1_per_class = np.mean(fold_f1s_per_class, axis=0)
            mean_f1_per_class_dict = {f'clase_{cl}': float(f1) for cl, f1 in zip(
                sorted(np.unique(labels)), mean_f1_per_class)}
            result = {
                'params': params,
                'mean_acc': mean_acc,
                'mean_f1_macro': mean_f1_macro,
                'std_acc': std_acc,
                'mean_f1_per_class': mean_f1_per_class_dict,
                'fold_results_acc': [float(x) for x in fold_accs],
                'fold_results_f1_macro': [float(x) for x in fold_f1_macros],
                'fold_results_f1_per_class': [f1.tolist() for f1 in fold_f1s_per_class]
            }
            prev_res["results"][params_key] = result
            with open(save_name, "w", encoding="utf-8") as f:
                json.dump(prev_res, f, indent=2, ensure_ascii=False)
        print(f"[OK] Resultados guardados en {save_name}")

### Orientación del sesgo

In [ ]:
MODEL_NAME = "PlanTL-GOB-ES/roberta-base-bne"
SEED = 42
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_CLASSES = 3

torch.manual_seed(SEED)
np.random.seed(SEED)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

news_loc = './csv/news.csv'
phrases_loc = './csv/processed_phrases_embed.csv'

pairs = obtener_parejas_berta(news_loc, phrases_loc)

grid_search_params = {
    'lr':        [2e-5, 5e-5],
    'dropout':   [0.1, 0.3],
    'epochs':    [5],
}
param_grid = list(ParameterGrid(grid_search_params))

BATCH_SIZE_CONFIG = {
    "news": 32,
    "phrases": 256
}
MAXLEN_CONFIG = {
    "news": 512,
    "phrases": 128
}

for i, (df, dataset_name) in enumerate(pairs):
    print(f"\n=========== Processing: {dataset_name} ===========")
    MAX_LEN = MAXLEN_CONFIG[dataset_name]
    BATCH_SIZE = BATCH_SIZE_CONFIG[dataset_name]
    df = df.fillna(0).reset_index(drop=True)

    def label_cat(x):
        if x > 0.5:
            return 1
        elif x < -0.5:
            return 2
        else:
            return 0
    df['label_cat'] = df['label'].apply(label_cat)
    texts = df['text'].astype(str).tolist()
    tabular_cols = [col for col in df.columns if col not in [
        'label', 'text', 'label_cat']]
    tabular_data = df[tabular_cols].values
    labels = df['label_cat'].values.astype(int)
    scaler = StandardScaler()
    tabular_scaled = scaler.fit_transform(tabular_data)
    dummy_tabular = np.zeros((len(df), 1), dtype=np.float32)

    for use_tabular in [True, False]:
        feature_type = "all" if use_tabular else "embedding"
        print(
            f"\n>> Entrenando con: {'todos los atributos' if use_tabular else 'solo embeddings de RoBERTa'}")
        X_tab = tabular_scaled if use_tabular else dummy_tabular
        kfold = KFold(n_splits=3, shuffle=True, random_state=SEED)
        save_name = f"resultados_{dataset_name}_undersampling_{feature_type}.json"
        if os.path.exists(save_name):
            try:
                with open(save_name, "r", encoding="utf-8") as f:
                    prev_res = json.load(f)
                already_done = set(prev_res["results"].keys())
                print(
                    f"[INFO] {save_name} encontrado, {len(already_done)}/{len(param_grid)} combinaciones ya guardadas.")
            except Exception as e:
                print(f"[WARN] Error leyendo {save_name}: {e}")
                already_done = set()
                prev_res = {"params_grid": param_grid, "results": {}}
        else:
            already_done = set()
            prev_res = {"params_grid": param_grid, "results": {}}
        for params in tqdm(param_grid, desc=f"Grid undersampling {feature_type}"):
            params_key = str(params)
            if params_key in already_done:
                continue
            fold_accs, fold_f1_macros, fold_f1s_per_class = [], [], []
            for fold_idx, (train_idx, val_idx) in enumerate(kfold.split(X_tab, labels)):
                train_texts = [texts[idx] for idx in train_idx]
                val_texts = [texts[idx] for idx in val_idx]
                train_tokenized = tokenizer(
                    train_texts,
                    add_special_tokens=True,
                    truncation=True,
                    max_length=MAX_LEN,
                    padding='max_length',
                    return_tensors='np'
                )
                val_tokenized = tokenizer(
                    val_texts,
                    add_special_tokens=True,
                    truncation=True,
                    max_length=MAX_LEN,
                    padding='max_length',
                    return_tensors='pt'
                )
                train_input_ids = train_tokenized['input_ids']
                train_attention_mask = train_tokenized['attention_mask']
                val_input_ids = val_tokenized['input_ids']
                val_attention_mask = val_tokenized['attention_mask']
                train_tabular, val_tabular = X_tab[train_idx], X_tab[val_idx]
                train_labels, val_labels = labels[train_idx], labels[val_idx]

                tabular_sampl, input_ids_sampl, att_mask_sampl, labels_sampl = undersample_multi(
                    train_tabular, train_labels, train_input_ids, train_attention_mask,
                    random_state=SEED
                )
                train_input_ids_sampl = torch.tensor(
                    input_ids_sampl, dtype=torch.long)
                train_attention_mask_sampl = torch.tensor(
                    att_mask_sampl, dtype=torch.long)
                tabular_sampl = tabular_sampl.astype(np.float32)

                train_dataset = TextTabularDataset(
                    train_input_ids_sampl, train_attention_mask_sampl, tabular_sampl, labels_sampl
                )
                val_dataset = TextTabularDataset(
                    val_input_ids, val_attention_mask, val_tabular, val_labels
                )
                train_loader = DataLoader(
                    train_dataset,
                    batch_size=BATCH_SIZE,
                    shuffle=True,
                    num_workers=4,
                    pin_memory=True
                )
                val_loader = DataLoader(
                    val_dataset,
                    batch_size=BATCH_SIZE,
                    shuffle=False,
                    num_workers=4,
                    pin_memory=True
                )
                present_classes = np.unique(labels_sampl)
                weights_present = compute_class_weight(
                    class_weight='balanced',
                    classes=present_classes,
                    y=labels_sampl
                )
                full_weights = np.zeros(NUM_CLASSES, dtype=np.float32)
                for cls, w in zip(present_classes, weights_present):
                    full_weights[int(cls)] = w
                fold_class_weights = torch.tensor(
                    full_weights, dtype=torch.float32, device=DEVICE)
                model = RobertaWithTabularDeepHead(
                    tabular_dim=train_tabular.shape[1],
                    num_classes=NUM_CLASSES,
                    dropout=params['dropout']
                )
                val_acc, val_f1_macro, val_f1_per_class = train_and_validate(
                    model,
                    train_loader,
                    val_loader,
                    class_weights=fold_class_weights,
                    epochs=params['epochs'],
                    lr=params['lr']
                )
                fold_accs.append(val_acc)
                fold_f1_macros.append(val_f1_macro)
                fold_f1s_per_class.append(val_f1_per_class)
            mean_acc = float(np.mean(fold_accs))
            mean_f1_macro = float(np.mean(fold_f1_macros))
            std_acc = float(np.std(fold_accs))
            mean_f1_per_class = np.mean(fold_f1s_per_class, axis=0)
            mean_f1_per_class_dict = {f'clase_{cl}': float(f1) for cl, f1 in zip(
                sorted(np.unique(labels)), mean_f1_per_class)}
            result = {
                'params': params,
                'mean_acc': mean_acc,
                'mean_f1_macro': mean_f1_macro,
                'std_acc': std_acc,
                'mean_f1_per_class': mean_f1_per_class_dict,
                'fold_results_acc': [float(x) for x in fold_accs],
                'fold_results_f1_macro': [float(x) for x in fold_f1_macros],
                'fold_results_f1_per_class': [f1.tolist() for f1 in fold_f1s_per_class]
            }
            prev_res["results"][params_key] = result
            with open(save_name, "w", encoding="utf-8") as f:
                json.dump(prev_res, f, indent=2, ensure_ascii=False)
        print(f"[OK] Resultados guardados en {save_name}")

## LightGBM

In [ ]:
def undersample_tabular(X, y, random_state=42):
    classes, counts = np.unique(y, return_counts=True)
    min_count = np.min(counts)
    sampling_strategy = {cls: min_count for cls in classes}
    rus = RandomUnderSampler(
        sampling_strategy=sampling_strategy, random_state=random_state)
    X_res, y_res = rus.fit_resample(X, y)
    return X_res, y_res

### tipo 5 de sesgo

In [ ]:
SEED = 42
NUM_CLASSES = 2


def procesar_tipos(ruta_phrases: str):
    df_ptype = pd.read_csv(ruta_phrases)
    df_ptype['bias_type'] = df_ptype['bias_type'].astype(str)
    labeltags = []
    for n in range(1, 6):
        col_name = f'label_{n}'
        labeltags.append(col_name)
        df_ptype[col_name] = df_ptype['bias_type'].apply(
            lambda x: str(n) in x.split(',')).astype(int)
    df_ptype = df_ptype.drop(columns=['idNew', 'idPhrase', 'bias_type'])
    df_ptype = df_ptype.fillna(0)
    return df_ptype, labeltags


phrases_loc = './csv/processed_phrases_embed.csv'
df_ptype, labeltags = procesar_tipos(phrases_loc)
first_emb = df_ptype.columns.get_loc('emb_0')
last_emb = df_ptype.columns.get_loc('emb_383')
df_noemb = df_ptype.drop(df_ptype.columns[first_emb:last_emb+1], axis=1)
X = df_noemb.drop(columns=labeltags)
tabular_cols = [col for col in X.columns if col != 'text']
text_col = 'text'
scaler = StandardScaler()
X_scaled = X.copy()
X_scaled[tabular_cols] = scaler.fit_transform(X_scaled[tabular_cols])
X_scaled[text_col] = X[text_col]

grid_search_params = {
    'learning_rate':   [0.05, 0.1],
    'num_leaves':      [15, 31],
    'max_depth':       [-1, 5],
    'n_estimators':    [100],
    'min_child_samples': [10],
    'subsample':       [1.0],
    'colsample_bytree': [1.0],
    'reg_alpha':       [0],
    'reg_lambda':      [0],
}
param_grid = list(ParameterGrid(grid_search_params))

labeltags = ["label_5"]
for labeltag in labeltags:
    print(f"\n=== Grid search LightGBM tabular para {labeltag} ===")
    y = df_ptype[labeltag].values.astype(int)
    df_actual = X_scaled.copy()
    df_actual['label'] = y
    tabular_data = df_actual[tabular_cols].values
    labels = y

    feature_type = "tabular"
    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=SEED)
    save_name = f"comparativa_{labeltag}_{feature_type}_undersampling_lightgbm.json"
    already_done = set()
    if os.path.exists(save_name):
        try:
            with open(save_name, "r", encoding="utf-8") as f:
                prev_res = json.load(f)
            already_done = set(prev_res["results"].keys())
            print(
                f"[INFO] {save_name} encontrado, {len(already_done)}/{len(param_grid)} combinaciones ya guardadas.")
        except Exception as e:
            print(f"[WARN] Error leyendo {save_name}: {e}")
            prev_res = {"params_grid": param_grid, "results": {}}
    else:
        prev_res = {"params_grid": param_grid, "results": {}}

    for params in tqdm(param_grid, desc=f"Grid undersampling LightGBM {feature_type} {labeltag}"):
        params_key = str(params)
        if params_key in already_done:
            continue
        fold_accs, fold_f1_macros, fold_f1s_per_class = [], [], []
        for fold_idx, (train_idx, val_idx) in enumerate(skf.split(tabular_data, labels)):
            X_train, y_train = tabular_data[train_idx], labels[train_idx]
            X_val, y_val = tabular_data[val_idx], labels[val_idx]
            X_train_bal, y_train_bal = undersample_tabular(
                X_train, y_train, random_state=SEED)
            clf = lgb.LGBMClassifier(
                device="gpu",
                objective='binary',
                random_state=SEED,
                verbose=-1,
                **params
            )
            clf.fit(X_train_bal, y_train_bal)
            val_preds = clf.predict(X_val)
            val_acc = accuracy_score(y_val, val_preds)
            val_f1_macro = f1_score(y_val, val_preds, average='macro')
            val_f1_per_class = f1_score(y_val, val_preds, average=None)
            fold_accs.append(val_acc)
            fold_f1_macros.append(val_f1_macro)
            fold_f1s_per_class.append(val_f1_per_class)
        if len(fold_accs) == 0:
            print(
                f"[WARN] Ningún fold válido para {params} en {labeltag} con {feature_type}")
            continue
        mean_acc = float(np.mean(fold_accs))
        mean_f1_macro = float(np.mean(fold_f1_macros))
        std_acc = float(np.std(fold_accs))
        mean_f1_per_class = np.mean(fold_f1s_per_class, axis=0)
        mean_f1_per_class_dict = {f'clase_{cl}': float(f1) for cl, f1 in zip(
            sorted(np.unique(labels)), mean_f1_per_class)}
        result = {
            'params': params,
            'mean_acc': mean_acc,
            'mean_f1_macro': mean_f1_macro,
            'std_acc': std_acc,
            'mean_f1_per_class': mean_f1_per_class_dict,
            'fold_results_acc': [float(x) for x in fold_accs],
            'fold_results_f1_macro': [float(x) for x in fold_f1_macros],
            'fold_results_f1_per_class': [f1.tolist() for f1 in fold_f1s_per_class]
        }
        prev_res["results"][params_key] = result
        with open(save_name, "w", encoding="utf-8") as f:
            json.dump(prev_res, f, indent=2, ensure_ascii=False)
    print(f"[OK] Resultados guardados en {save_name}")

### orientación de sesgo (documento)

In [ ]:
SEED = 42

news_loc = './csv/news.csv'
phrases_loc = './csv/processed_phrases_embed.csv'


def obtener_parejas_tabular(news_path, phrases_path):
    df_news = pd.read_csv(news_path).fillna(0)
    df_phrases = pd.read_csv(phrases_path).fillna(0)
    drop_news = ['idNew', 'fecha', 'newspaper', 'count_X',
                 'count_X_log', 'ratio_words_X', 'ratio_sentences_X']
    first_emb_news = df_news.columns.get_loc('emb_0')
    last_emb_news = df_news.columns.get_loc('emb_383')
    emb_cols_news = df_news.columns[first_emb_news:last_emb_news+1]
    df_news = df_news.drop(columns=drop_news +
                           list(emb_cols_news), errors='ignore')
    drop_phrases = ['idNew', 'idPhrase', 'bias_type']
    first_emb_phrases = df_phrases.columns.get_loc('emb_0')
    last_emb_phrases = df_phrases.columns.get_loc('emb_383')
    emb_cols_phrases = df_phrases.columns[first_emb_phrases:last_emb_phrases+1]
    df_phrases = df_phrases.drop(
        columns=drop_phrases + list(emb_cols_phrases), errors='ignore')

    def label_cat(x):
        if x > 0.5:
            return 1
        elif x < -0.5:
            return 2
        else:
            return 0
    df_phrases['label_cat'] = df_phrases['bias'].apply(label_cat)
    df_news['label_cat'] = df_news['label'].apply(label_cat)
    pairs = [(df_news, "news")]
    return pairs


grid_search_params = {
    'learning_rate':   [0.05, 0.1],
    'num_leaves':      [15, 31],
    'max_depth':       [-1, 5],
    'n_estimators':    [100],
    'min_child_samples': [10],
    'subsample':       [1.0],
    'colsample_bytree': [1.0],
    'reg_alpha':       [0],
    'reg_lambda':      [0],
}
param_grid = list(ParameterGrid(grid_search_params))

pairs = obtener_parejas_tabular(news_loc, phrases_loc)

for df, dataset_name in pairs:
    print(f"\n=== Grid search LightGBM tabular para {dataset_name} ===")
    y = df['label_cat'].values.astype(int)
    tabular_cols = [col for col in df.columns if col not in [
        'label', 'label_cat', 'text']]
    X = df[tabular_cols].values
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    kfold = KFold(n_splits=3, shuffle=True, random_state=SEED)
    save_name = f"comparativa_{dataset_name}_tabular_undersampling_lightgbm.json"
    already_done = set()
    if os.path.exists(save_name):
        try:
            with open(save_name, "r", encoding="utf-8") as f:
                prev_res = json.load(f)
            already_done = set(prev_res["results"].keys())
            print(
                f"[INFO] {save_name} encontrado, {len(already_done)}/{len(param_grid)} combinaciones ya guardadas.")
        except Exception as e:
            print(f"[WARN] Error leyendo {save_name}: {e}")
            prev_res = {"params_grid": param_grid, "results": {}}
    else:
        prev_res = {"params_grid": param_grid, "results": {}}

    for params in tqdm(param_grid, desc=f"Grid undersampling LightGBM tabular {dataset_name}"):
        params_key = str(params)
        if params_key in already_done:
            continue
        fold_accs, fold_f1_macros, fold_f1s_per_class = [], [], []
        for fold_idx, (train_idx, val_idx) in enumerate(kfold.split(X_scaled, y)):
            X_train, y_train = X_scaled[train_idx], y[train_idx]
            X_val, y_val = X_scaled[val_idx], y[val_idx]
            X_train_bal, y_train_bal = undersample_tabular(
                X_train, y_train, random_state=SEED)
            clf = lgb.LGBMClassifier(
                device="gpu",
                objective='multiclass',
                num_class=3,
                random_state=SEED,
                verbose=-1,
                **params
            )
            clf.fit(X_train_bal, y_train_bal)
            val_preds = clf.predict(X_val)
            val_acc = accuracy_score(y_val, val_preds)
            val_f1_macro = f1_score(y_val, val_preds, average='macro')
            val_f1_per_class = f1_score(y_val, val_preds, average=None)
            fold_accs.append(val_acc)
            fold_f1_macros.append(val_f1_macro)
            fold_f1s_per_class.append(val_f1_per_class)
        mean_acc = float(np.mean(fold_accs))
        mean_f1_macro = float(np.mean(fold_f1_macros))
        std_acc = float(np.std(fold_accs))
        mean_f1_per_class = np.mean(fold_f1s_per_class, axis=0)
        mean_f1_per_class_dict = {f'clase_{cl}': float(
            f1) for cl, f1 in enumerate(mean_f1_per_class)}
        result = {
            'params': params,
            'mean_acc': mean_acc,
            'mean_f1_macro': mean_f1_macro,
            'std_acc': std_acc,
            'mean_f1_per_class': mean_f1_per_class_dict,
            'fold_results_acc': [float(x) for x in fold_accs],
            'fold_results_f1_macro': [float(x) for x in fold_f1_macros],
            'fold_results_f1_per_class': [f1.tolist() for f1 in fold_f1s_per_class]
        }
        prev_res["results"][params_key] = result
        with open(save_name, "w", encoding="utf-8") as f:
            json.dump(prev_res, f, indent=2, ensure_ascii=False)
    print(f"[OK] Resultados guardados en {save_name}")